In [6]:
import os
import shutil
import random
from pathlib import Path

# Set paths
original_dataset_dir = Path(r"C:\Users\kdeep\p2\renu\baby2")
base_dir = Path("C:/Users/kdeep/p2/renu/baby_split")
classes = ['on_stomach', 'on_back', 'side']

# Create base folders
for split in ['train', 'val', 'test']:
    for cls in classes:
        (base_dir / split / cls).mkdir(parents=True, exist_ok=True)

# Split ratios
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

# Go through each class folder
for cls in classes:
    files = [f for f in (original_dataset_dir / cls).iterdir() if f.suffix.lower() in ['.jpg', '.jpeg', '.png']]

    random.shuffle(files)

    total = len(files)
    train_end = int(train_ratio * total)
    val_end = train_end + int(val_ratio * total)

    splits = {
        'train': files[:train_end],
        'val': files[train_end:val_end],
        'test': files[val_end:]
    }

    # Copy files
    for split, split_files in splits.items():
        for f in split_files:
            shutil.copy(f, base_dir / split / cls / f.name)

print("✅ Dataset split completed!")


✅ Dataset split completed!


In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import os

# Set paths
base_dir = "C:/Users/kdeep/p2/renu/baby_split"
img_size = (224, 224)
batch_size = 16

# Data generators
train_gen = ImageDataGenerator(rescale=1./255, rotation_range=20, zoom_range=0.2, horizontal_flip=True)
val_gen = ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_directory(
    os.path.join(base_dir, 'train'),
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'  # ✅ Now supports 3 classes
)

val_data = val_gen.flow_from_directory(
    os.path.join(base_dir, 'val'),
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Build model
model = models.Sequential([
    layers.Input(shape=(224, 224, 3)),
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(3, activation='softmax')  # ✅ 3 output classes
])

# Compile
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train
history = model.fit(train_data, validation_data=val_data, epochs=10)

# Save model
model.save("baby_sleep_position_3class_model.h5")


Found 153 images belonging to 3 classes.
Found 31 images belonging to 3 classes.
Epoch 1/10


C:\Users\kdeep\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


10/10 ━━━━━━━━━━━━━━━━━━━━ 16s 898ms/step - accuracy: 0.2966 - loss: 3.3704 - val_accuracy: 0.5484 - val_loss: 1.0185
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 7s 452ms/step - accuracy: 0.5409 - loss: 1.0318 - val_accuracy: 0.5484 - val_loss: 1.0315
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 7s 492ms/step - accuracy: 0.5117 - loss: 1.0159 - val_accuracy: 0.5484 - val_loss: 1.0272
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 7s 474ms/step - accuracy: 0.5560 - loss: 0.9651 - val_accuracy: 0.5484 - val_loss: 1.0160
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 7s 467ms/step - accuracy: 0.5081 - loss: 0.9722 - val_accuracy: 0.5806 - val_loss: 1.0106
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 7s 472ms/step - accuracy: 0.5955 - loss: 0.8878 - val_accuracy: 0.5806 - val_loss: 0.9622
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 7s 443ms/step - accuracy: 0.6223 - loss: 0.8576 - val_accuracy: 0.4194 - val_loss: 1.1573
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 7s 449ms/step - accuracy: 0.6115 - loss: 0.9041 - val_accuracy: 0.5484 - val

In [8]:
test_gen = ImageDataGenerator(rescale=1./255)
test_data = test_gen.flow_from_directory(
    os.path.join(base_dir, 'test'),
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',   # ✅ 3-class mode
    shuffle=False               # ✅ Optional: ensures correct label order
)

loss, acc = model.evaluate(test_data)
print(f"✅ Test Accuracy: {acc * 100:.2f}%")



Found 36 images belonging to 3 classes.
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - accuracy: 0.6962 - loss: 0.8835 
✅ Test Accuracy: 61.11%


In [9]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image

# Load model
model = tf.keras.models.load_model("baby_sleep_position_3class_model.h5")
class_names = ['on_back', 'on_stomach', 'side']

# Preprocessing and prediction
def predict_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)[0]
    predicted_class = class_names[np.argmax(prediction)]
    return predicted_class

# Browse and predict
def browse_and_predict():
    file_path = filedialog.askopenfilename()
    if file_path:
        img = Image.open(file_path)
        img_resized = img.resize((250, 250))
        img_tk = ImageTk.PhotoImage(img_resized)
        panel.config(image=img_tk)
        panel.image = img_tk

        result = predict_image(file_path)
        result_label.config(text=f"Prediction: {result}")

# GUI Setup
root = tk.Tk()
root.title("Sleep Position Classifier")

panel = tk.Label(root)
panel.pack()

btn = tk.Button(root, text="Browse Image", command=browse_and_predict)
btn.pack()

result_label = tk.Label(root, text="", font=("Arial", 16))
result_label.pack()

root.mainloop()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
